In [1]:
import os
os.chdir('/data/hyeryung/mucoco')

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from new_module.locate.new_locate_utils import LocateMachine
import pandas as pd

In [42]:
import new_module.locate.new_locate_utils
import importlib
importlib.reload(new_module.locate.new_locate_utils)
from new_module.locate.new_locate_utils import LocateMachine

In [2]:
prompt='abc'
prediction=['dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe']
ckpt_path = '/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint/'
model = AutoModelForSequenceClassification.from_pretrained(ckpt_path)
tokenizer = AutoTokenizer.from_pretrained(ckpt_path)
device='cuda'
model = model.to(device)

In [3]:
%%timeit
loc_machine=LocateMachine(model,tokenizer)
res = loc_machine.locate_main(prediction, "attention", max_num_tokens = 6, unit="word", num_layer=10, label_id=0, 
                              num_processes=4)

8.41 s ± 595 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
loc_machine=LocateMachine(model,tokenizer)
res = loc_machine.locate_main(prediction, "attention", max_num_tokens = 6, unit="word", num_layer=10, label_id=0, 
                              num_processes=2)

7.94 s ± 282 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%%timeit
## when modified the code to run a for loop
loc_machine=LocateMachine(model,tokenizer)
res = loc_machine.locate_main(prediction, "attention", max_num_tokens = 6, unit="word", num_layer=10, label_id=0, 
                              num_processes=1)

31.6 ms ± 4.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
def unravel(outputs_df):
    outputs_df=outputs_df.explode('generations',ignore_index=True)
    outputs_df['prompt']=outputs_df['prompt'].apply(lambda x: x['text'])
    outputs_df['generations']=outputs_df['generations'].apply(lambda x: x['text'] if isinstance(x, dict) else x)
    outputs_df = outputs_df.dropna().reset_index(drop=True)
    return outputs_df
toxic_gen=pd.read_json('new_module/data/toxicity-avoidance/dev_set.jsonl',lines=True)
toxic_gen=unravel(toxic_gen)
prompt = '“I went to use the bathroom, and I went in and Lance was bent'
prediction = toxic_gen.loc[toxic_gen['prompt']==prompt,'generations'].tolist()
ckpt_path = '/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint/'
model = AutoModelForSequenceClassification.from_pretrained(ckpt_path)
tokenizer = AutoTokenizer.from_pretrained(ckpt_path)
device='cuda'
model = model.to(device)

In [14]:
%%timeit
## when modified the code to run a for loop
loc_machine=LocateMachine(model,tokenizer)
res = loc_machine.locate_main(prediction, "attention", max_num_tokens = 6, unit="word", num_layer=10, label_id=0, 
                              num_processes=1)

37.1 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%timeit
loc_machine=LocateMachine(model,tokenizer)
res = loc_machine.locate_main(prediction, "attention", max_num_tokens = 6, unit="word", num_layer=10, label_id=0, 
                              num_processes=2)

8.92 s ± 1.37 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
## with for-loop implementation
## after changing the function to return masked sequences (text) instead of located_indices
loc_machine=LocateMachine(model,tokenizer)
res = loc_machine.locate_main(prediction, "attention", max_num_tokens = 6, unit="word", num_layer=10, label_id=0)

40.5 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
